Routines for plotting the short-term performance statistics (e.g., RMSE/ACC/Spectra) of the emulator, averaged across different randomly initialized and trained emulators.

In [1]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import yaml
from analysis.plot_config import params

In [ ]:
# List all ensemble run_nums

#run_nums = ['BASE_seed_0_2phase_1000epochs'] #,
#            'BASE_seed_0_2phase_1000epochs_ic1',
#            'BASE_seed_0_2phase_1000epochs_ic2',
#            'BASE_seed_0_2phase_1000epochs_ic3',
#            'BASE_seed_0_2phase_1000epochs_ic4']

#run_nums = ['MAE_mr0.5_seed_0_2phase_ft_dropPR'] #,
#            'MAE_mr0.5_seed_0_2phase_ft_dropPR_ic1',
#            'MAE_mr0.5_seed_0_2phase_ft_dropPR_ic2',
#            'MAE_mr0.5_seed_0_2phase_ft_dropPR_ic3',
#            'MAE_mr0.5_seed_0_2phase_ft_dropPR_ic4']

run_nums = ['SMAE_seed_0_2phase_spatial_pixelonly_targetfull_0031_ft_dropPR',
            'SMAE_seed_0_2phase_spatial_pixelonly_targetfull_0031_ft_dropPR_ic1',
            'SMAE_seed_0_2phase_spatial_pixelonly_targetfull_0031_ft_dropPR_ic2',
            'SMAE_seed_0_2phase_spatial_pixelonly_targetfull_0031_ft_dropPR_ic3',
            'SMAE_seed_0_2phase_spatial_pixelonly_targetfull_0031_ft_dropPR_ic4']

# Config file path
config_file_path = "config/config_SMAE_seed_0_2phase_spatial_pixelonly_targetfull_0031_ft_dropPR.yaml"
with open(config_file_path, "r") as f:
    config = yaml.safe_load(f)

dataset_params = config["dataset_params"]
short_analysis_params = config["short_analysis_params"]
long_analysis_params = config["long_analysis_params"]

root_dir = dataset_params["root_dir"]

# Saving plot data
save_plot_data = True
curves_save_dir = os.path.join('/glade/u/home/dpatel', run_nums[0])
print(f'curves_save_dir location: {curves_save_dir}')

curves_save_dir location: /glade/u/home/dpatel/SMAE_seed_0_2phase_spatial_pixelonly_targetfull_0031_ft_dropPR


In [ ]:
# Gather all of the data

rmse_u, rmse_v = [], []
acc_u, acc_v = [], []
spectra_pred, spectra_tar = [], []

for run_num in run_nums:
    analysis_dir = os.path.join(root_dir, run_num, "analysis")
    short_analysis_results_fp = os.path.join(analysis_dir, "emulate", "short_analysis.npz")

    data = np.load(short_analysis_results_fp)

    rmse_u.append(data["rmse_u"])
    rmse_v.append(data["rmse_v"])
    acc_u.append(data["acc_u"])
    acc_v.append(data["acc_v"])
    spectra_pred.append(data["spectra"])
    spectra_tar = data["spectra_tar"]
    wavenumbers = data["wavenumbers"]

    rmse_u_per_median = data["rmse_u_per_median"]
    rmse_u_per_uq = data["rmse_u_per_uq"]
    rmse_u_per_lq = data["rmse_u_per_lq"]
    rmse_v_per_lq = data["rmse_v_per_lq"]
    rmse_v_per_median = data["rmse_v_per_median"]
    rmse_v_per_uq = data["rmse_v_per_uq"]
    rmse_v_per_lq = data["rmse_v_per_lq"]

    acc_u_per_median = data["acc_u_per_median"]
    acc_u_per_uq = data["acc_u_per_uq"]
    acc_u_per_lq = data["acc_u_per_lq"]
    acc_v_per_lq = data["acc_v_per_lq"]
    acc_v_per_median = data["acc_v_per_median"]
    acc_v_per_uq = data["acc_v_per_uq"]
    acc_v_per_lq = data["acc_v_per_lq"]

rmse_u = np.concatenate(rmse_u, axis=0)
rmse_v = np.concatenate(rmse_v, axis=0)
acc_u = np.concatenate(acc_u, axis=0)
acc_v = np.concatenate(acc_v, axis=0)
spectra_pred = np.stack(spectra_pred, axis=0)  # shape (len(run_nums), leadtimes, wavenumbers)

print(f'rmse_u.shape: {rmse_u.shape}')
print(f'acc_u.shape: {acc_u.shape}')
print(f'spectra_pred.shape: {spectra_pred.shape}')

Root-Mean-Square-Error

In [ ]:
#font = {'size'  : 16}
#mpl.rc('font', **font)
plt.rcParams.update(params)

rmse_u_median = np.quantile(rmse_u, 0.5, axis=0)
rmse_u_uq = np.quantile(rmse_u, 0.75, axis=0)
rmse_u_lq = np.quantile(rmse_u, 0.25, axis=0)

fig, ax = plt.subplots(figsize=(4, 4))
x = np.arange(1, len(rmse_u_median)+1) 
ax.plot(x, rmse_u_median, '-b', label='Emulator')
ax.fill_between(x, rmse_u_lq, rmse_u_uq, color='b', alpha=0.1)
ax.plot(x, rmse_u_per_median, '--k', label='Persistence')
ax.fill_between(x, rmse_u_per_lq, rmse_u_per_uq, color='k', alpha=0.1)
ax.set_ylabel('RMSE')
ax.set_xlabel(rf'Lead time ($\Delta t$)')
ax.set_ylim([0, 3.5])
ax.set_xlim([0, len(rmse_u_median)+1])

#for i in range(5):
#    ax.plot(x, np.median(rmse_u[i*200:i*200+200], axis=0), linewidth=0.5)

# Save data
if save_plot_data:
    np.savez(os.path.join(curves_save_dir, f'rmse_U.npz'),
             x=x,
             rmse_u_median=rmse_u_median, rmse_u_per_median=rmse_u_per_median,
             rmse_u_lq=rmse_u_lq, rmse_u_per_lq=rmse_u_per_lq,
             rmse_u_uq=rmse_u_uq, rmse_u_per_uq=rmse_u_per_uq)

ax.legend(frameon=False)
plt.tight_layout()

In [ ]:
#font = {'size'  : 16}
#pl.rc('font', **font)
plt.rcParams.update(params)

rmse_v_median = np.quantile(rmse_v, 0.5, axis=0)
rmse_v_uq = np.quantile(rmse_v, 0.75, axis=0)
rmse_v_lq = np.quantile(rmse_v, 0.25, axis=0)

fig, ax = plt.subplots(figsize=(4, 4))
x = np.arange(1, len(rmse_v_median)+1) 
ax.plot(x, rmse_v_median, '-b', label='Emulator')
ax.fill_between(x, rmse_v_lq, rmse_v_uq, color='b', alpha=0.1)
ax.plot(x, rmse_v_per_median, '--k', label='Persistence')
ax.fill_between(x, rmse_v_per_lq, rmse_v_per_uq, color='k', alpha=0.1)
ax.set_ylabel('RMSE')
ax.set_xlabel(rf'Lead time ($\Delta t$)')
ax.set_ylim([0, 3.5])
ax.set_xlim([0, len(rmse_v_median)+1])

#for i in range(5):
#    ax.plot(x, np.median(rmse_v[i*200:i*200+200], axis=0), linewidth=0.5)

ax.legend(frameon=False)
plt.tight_layout()

Anomaly Correlation Coefficient

In [ ]:
#font = {'size'  : 16}
#mpl.rc('font', **font)
plt.rcParams.update(params)

acc_u_median = np.quantile(acc_u, 0.5, axis=0)
acc_u_uq = np.quantile(acc_u, 0.75, axis=0)
acc_u_lq = np.quantile(acc_u, 0.25, axis=0)

fig, ax = plt.subplots(figsize=(4, 4))
x = np.arange(1, len(acc_u_median)+1) 
ax.plot(x, acc_u_median, '-b', label='Emulator')
ax.fill_between(x, acc_u_lq, acc_u_uq, color='b', alpha=0.1)
ax.plot(x, acc_u_per_median, '--k', label='Persistence')
ax.fill_between(x, acc_u_per_lq, acc_u_per_uq, color='k', alpha=0.1)
ax.set_ylabel('ACC')
ax.set_xlabel(rf'Lead time ($\Delta t$)')
ax.set_ylim([0, 1])
ax.set_xlim([0, len(acc_u_median)])

# Save data
if save_plot_data:
    np.savez(os.path.join(curves_save_dir, f'acc_U.npz'),
             x=x,
             acc_u_median=acc_u_median, acc_u_per_median=acc_u_per_median,
             acc_u_lq=acc_u_lq, acc_u_per_lq=acc_u_per_lq,
             acc_u_uq=acc_u_uq, acc_u_per_uq=acc_u_per_uq)

#for i in range(5):
#   ax.plot(x, np.median(acc_u[i*200:i*200+200], axis=0), linewidth=0.5)

#ax.legend(frameon=False)
plt.tight_layout()

In [ ]:
#font = {'size'  : 16}
#mpl.rc('font', **font)
plt.rcParams.update(params)

acc_v_median = np.quantile(acc_v, 0.5, axis=0)
acc_v_uq = np.quantile(acc_v, 0.75, axis=0)
acc_v_lq = np.quantile(acc_v, 0.25, axis=0)

fig, ax = plt.subplots(figsize=(4, 4))
x = np.arange(1, len(acc_v_median)+1) 
ax.plot(x, acc_v_median, '-b', label='Emulator')
ax.fill_between(x, acc_v_lq, acc_u_uq, color='b', alpha=0.1)
ax.plot(x, acc_v_per_median, '--k', label='Persistence')
ax.fill_between(x, acc_v_per_lq, acc_v_per_uq, color='k', alpha=0.1)
ax.set_ylabel('ACC')
ax.set_xlabel(rf'Lead time ($\Delta t$)')
ax.set_ylim([0, 1])
ax.set_xlim([0, len(acc_v_median)])

#for i in range(5):
#   ax.plot(x, np.median(acc_v[i*200:i*200+200], axis=0), linewidth=0.5)

#ax.legend()
plt.tight_layout()

Spectra

In [ ]:
# font = {'size'  : 16}
# mpl.rc('font', **font)
plt.rcParams.update(params)

spectra_pred_mean = np.mean(spectra_pred, axis=0)
spectra_tar_mean = np.mean(spectra_tar, axis=0)

leadtimes = short_analysis_params["spectra_leadtimes"]

fig, ax = plt.subplots(figsize=(4, 4))
ax.plot(wavenumbers, spectra_tar_mean, '-k', label='Truth')
for lead in leadtimes:
    spec = spectra_pred_mean[lead]

    label = rf'{lead+1}$\Delta t$' 

    ax.plot(wavenumbers, spec, label=label)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Wavenumbers')
    ax.set_ylabel('Power')
    ax.set_xlim([0.8, 200])
    ax.set_ylim([10**(-9), 10])
    ax.legend()

plt.tight_layout()

In [ ]:
# font = {'size'  : 16}
# mpl.rc('font', **font)
plt.rcParams.update(params)

spectra_pred_median = np.median(spectra_pred, axis=0)
spectra_pred_uq = np.quantile(spectra_pred, 0.75, axis=0)
spectra_pred_lq = np.quantile(spectra_pred, 0.25, axis=0)
spectra_tar_median = np.median(spectra_tar, axis=0)
spectra_tar_uq = np.quantile(spectra_tar, 0.75, axis=0)
spectra_tar_lq = np.quantile(spectra_tar, 0.25, axis=0)

leadtimes = short_analysis_params["spectra_leadtimes"]

fig, ax = plt.subplots(figsize=(4, 4))
ax.plot(wavenumbers, spectra_tar_median, '-k', label='Truth')
ax.plot(wavenumbers, spectra_tar_lq, spectra_tar_uq, color='k', alpha=0.1)
for lead in leadtimes:
    spec = spectra_pred_median[lead]

    label = rf'{lead+1}$\Delta t$' 

    ax.plot(wavenumbers, spec, label=label)
    ax.fill_between(wavenumbers, spectra_pred_lq[lead], spectra_pred_uq[lead], alpha=0.5)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel('Wavenumbers')
    ax.set_ylabel('Power')
    ax.set_xlim([0.8, 200])
    ax.set_ylim([1e-8, 1e1])
    ax.legend(frameon=False)

plt.tight_layout()